### Oracle - cx_Oracle

#### 1. Connecting to Oracle

In [1]:
!pip install cx_oracle

In [2]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle

# 오라클 서비스 시작 후 실행
con = cx_Oracle.connect(user='scott', password='tiger', dsn='localhost:1522/orcl')

cur = con.cursor()   # SQL 수행 결과를 담을 메모리 공간
cur.execute("""
            SELECT *
            FROM DEPT
        """)
res = cur.fetchall()  # 커서에 담긴 내용을 모두 꺼내옴
for row in res:
    print(row)


(10, 'ACCOUNTING', 'NEW YORK')
(20, 'RESEARCH', 'DALLAS')
(30, 'SALES', 'CHICAGO')
(40, 'OPERATIONS', 'BOSTON')


#### 2. Connection Pooling

In [3]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


def Query():
    con = pool.acquire()
    cur = con.cursor()
    for i in range(4):
        cur.execute(f"""
                    SELECT {i}
                      FROM DUAL
                """)
                
        seqval, = cur.fetchone()
        print("Thread",
              threading.current_thread().name,
              "fetched sequence =", seqval)


thread1 = threading.Thread(name='#1', target=Query)
thread1.start()

thread2 = threading.Thread(name='#2', target=Query)
thread2.start()

thread1.join()
thread2.join()

print("All done!")


ThreadThread #1 fetched sequence = 0
 #2 fetched sequence = 0
Thread #2 fetched sequence = 1
Thread #2 fetched sequence = 2
Thread #2 fetched sequence = 3
Thread #1 fetched sequence = 1
Thread #1 fetched sequence = 2
Thread #1 fetched sequence = 3
All done!


#### 스레딩 연습

In [4]:
# thread_test.py
import time

def long_task():  # 5초의 시간이 걸리는 함수
    for i in range(5):
        time.sleep(1)  # 1초간 대기한다.
        print("working:%s\n" % i)

print("Start")

for i in range(5):  # long_task를 5회 수행한다.
    long_task()

print("End")

Start
working:0

working:1

working:2

working:3

working:4

working:0

working:1

working:2

working:3

working:4

working:0

working:1

working:2

working:3

working:4

working:0

working:1

working:2

working:3

working:4

working:0

working:1

working:2

working:3

working:4

End


In [6]:
import time
import threading  # 스레드를 생성하기 위해서는 threading 모듈이 필요하다.

def long_task():
    for i in range(5):
        time.sleep(1)
        print("working:%s\n" % i)

print("Start")

threads = []
for i in range(5):
    t = threading.Thread(target=long_task)  # 스레드를 생성한다.
    threads.append(t) 

for t in threads:
    t.start()  # 스레드를 실행한다.

for t in threads:
    t.join()  # join으로 스레드가 종료될때까지 기다린다.
    
print("End")

Start
working:0

working:0
working:0


working:0

working:0

working:1

working:1

working:1
working:1


working:1

working:2

working:2

working:2
working:2


working:2

working:3

working:3
working:3

working:3


working:3

working:4

working:4

working:4
working:4


working:4

End


#### 3. Fetching Data

##### 3.1 A simple query

In [7]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

cur.execute("""
            SELECT *
              FROM DEPT
            ORDER BY DEPTNO
        """)

for deptno, dname, loc in cur:
    print("Department number: ", deptno)
    print("Department name: ", dname)
    print("Department location:", loc)
    print()


Department number:  10
Department name:  ACCOUNTING
Department location: NEW YORK

Department number:  20
Department name:  RESEARCH
Department location: DALLAS

Department number:  30
Department name:  SALES
Department location: CHICAGO

Department number:  40
Department name:  OPERATIONS
Department location: BOSTON



##### 3.2 Using fetchone()

In [8]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

cur.execute("""
            SELECT *
              FROM DEPT
            ORDER BY DEPTNO
        """)

row = cur.fetchone()
print(row)

row = cur.fetchone()
print(row)


(10, 'ACCOUNTING', 'NEW YORK')
(20, 'RESEARCH', 'DALLAS')


##### 3.3 Using fetchmany()

In [10]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

cur.execute("""
            SELECT *
              FROM DEPT
            ORDER BY DEPTNO
        """)

res = cur.fetchmany(numRows = 3)
print(res)
print()
print(res[0])    # first row
print()
print(res[0][1]) # second element of first row


[(10, 'ACCOUNTING', 'NEW YORK'), (20, 'RESEARCH', 'DALLAS'), (30, 'SALES', 'CHICAGO')]

(10, 'ACCOUNTING', 'NEW YORK')

ACCOUNTING


#### 4. Binding Data

##### 4.1 Binding in queries

In [12]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

sql = """
    SELECT *
      FROM DEPT
     WHERE DEPTNO = :id
    ORDER BY DEPTNO
"""

cur.execute(sql, id = 20)
res = cur.fetchall()
print(res)

cur.execute(sql, id = 10)
res = cur.fetchall()
print(res)


[(20, 'RESEARCH', 'DALLAS')]
[(10, 'ACCOUNTING', 'NEW YORK')]


##### 4.2 Binding in inserts

In [13]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

# Create temporary table
cur.execute("DROP TABLE MYTAB")
cur.execute("CREATE TABLE MYTAB (ID NUMBER, DATA VARCHAR2(100) UNIQUE)")

rows = [ (1, "First" ),
         (2, "Second" ),
         (3, "Third" ),
         (4, "Fourth" ),
         (5, "Fifth" ),
         (6, "Sixth" ),
         (7, "Seventh" ) ]

# Insert data
cur.executemany("""
                INSERT INTO MYTAB (id, data)
                           VALUES (:1, :2)
                """
                , rows)

# con.commit()
# con.rollback()

# Now query the results back
cur.execute("SELECT * FROM MYTAB")
res = cur.fetchall()
print(res)


[(1, 'First'), (2, 'Second'), (3, 'Third'), (4, 'Fourth'), (5, 'Fifth'), (6, 'Sixth'), (7, 'Seventh')]


##### 4.3 Batcherrors

In [14]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

# Create temporary table
cur.execute("DROP TABLE MYTAB")
cur.execute("CREATE TABLE MYTAB (ID NUMBER, DATA VARCHAR2(100) UNIQUE)")

rows = [ (1, "First" ), (2, "Second" ),
         (3, "Third" ), (4, "Fourth" ),
         (5, "Third" ), (6, "Sixth" ),
         (7, "Seventh" ) ]

# Insert data
cur.executemany("""
                INSERT INTO MYTAB (id, data)
                           VALUES (:1, :2)
                """
                , rows
                , batcherrors=True)

for error in cur.getbatcherrors():
    print("Error", error.message.rstrip(), "at row offset", error.offset)

# con.commit()
# con.rollback()

# Now query the results back
cur.execute("SELECT * FROM MYTAB")
res = cur.fetchall()
print(res)


Error ORA-00001: 무결성 제약 조건(SCOTT.SYS_C0011161)에 위배됩니다 at row offset 4
[(1, 'First'), (2, 'Second'), (3, 'Third'), (4, 'Fourth'), (6, 'Sixth'), (7, 'Seventh')]


#### DB - Pandas 연동

In [16]:
'''
https://oracle.github.io/python-cx_Oracle
https://oracle.github.io/python-cx_Oracle/samples/tutorial/Python-and-Oracle-Database-Scripting-for-the-Future.html
'''

import cx_Oracle    # pip install cx_oracle
import threading
import pandas as pd


pool = cx_Oracle.SessionPool('scott', 'tiger', 'localhost:1522/orcl',
                             min = 2, max = 5, increment = 1, threaded = True,
                             getmode = cx_Oracle.SPOOL_ATTRVAL_WAIT)


con = pool.acquire()
cur = con.cursor()

# Create temporary table
cur.execute("DROP TABLE MYTAB")
cur.execute("CREATE TABLE MYTAB (ID NUMBER, DATA VARCHAR2(100) UNIQUE)")

rows = [ (1, "First" ), (2, "Second" ),
         (3, "Third" ), (4, "Fourth" ),
         (5, "Fifth" ), (6, "Sixth" ),
         (7, "Seventh" ) ]

# Insert data
cur.executemany("""
                INSERT INTO MYTAB (id, data)
                           VALUES (:1, :2)
                """
                , rows)

# con.commit()
# con.rollback()

# Now query the results back
cur.execute("SELECT * FROM MYTAB")
res = cur.fetchall()
print(res)

df = pd.read_sql("SELECT * FROM MYTAB", con)
df


[(1, 'First'), (2, 'Second'), (3, 'Third'), (4, 'Fourth'), (5, 'Fifth'), (6, 'Sixth'), (7, 'Seventh')]


,ID,DATA
0,1,First
1,2,Second
2,3,Third
3,4,Fourth
4,5,Fifth
5,6,Sixth
6,7,Seventh


### SQLite

- https://docs.python.org/ko/3/library/sqlite3.html
- https://wikidocs.net/book/1530

#### 1. Connection

In [1]:
'''
https://docs.python.org/ko/3/library/sqlite3.html
'''

import sqlite3


con = sqlite3.connect('example.db')


#### 2. Data

In [3]:
'''
https://docs.python.org/ko/3/library/sqlite3.html
'''

import sqlite3


con = sqlite3.connect('example.db')
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE STOCKS
               (DATE TEXT, TRANS TEXT, SYMBOL TEXT, QTY REAL, PRICE REAL)''')

# Insert a row of data
cur.execute("INSERT INTO STOCKS VALUES ('2006-01-05', 'BUY', 'RHAT', 100, 35.14)")

# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()


OperationalError: table STOCKS already exists

#### 2. Selection

In [4]:
'''
https://docs.python.org/ko/3/library/sqlite3.html
'''

import sqlite3


con = sqlite3.connect('example.db')
cur = con.cursor()

for row in cur.execute('SELECT * FROM STOCKS ORDER BY PRICE'):
    print(row)


('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


#### 3. Database in Memory

In [8]:
'''
https://docs.python.org/ko/3/library/sqlite3.html
'''

import sqlite3


con = sqlite3.connect(":memory:")
cur = con.cursor()

cur.execute("CREATE TABLE LANG (NAME, FIRST_APPEARED)")

# This is the qmark style:
cur.execute("INSERT INTO LANG VALUES (?, ?)", ("C", 1972))

# The qmark style used with executemany():
lang_list = [
    ("Fortran", 1957),
    ("Python", 1991),
    ("Go", 2009),
]
cur.executemany("INSERT INTO LANG VALUES (?, ?)", lang_list)

# And this is the named style:
cur.execute("SELECT * FROM LANG WHERE FIRST_APPEARED=:year", {"year": 1972})
print(cur.fetchall())

con.close()


[('C', 1972)]


#### DB - Pandas 연동

In [10]:
'''
https://docs.python.org/ko/3/library/sqlite3.html
'''

import sqlite3
import pandas as pd


con = sqlite3.connect(":memory:")
cur = con.cursor()

cur.execute("CREATE TABLE LANG (NAME, FIRST_APPEARED)")

# This is the qmark style:
cur.execute("INSERT INTO LANG VALUES (?, ?)", ("C", 1972))

# The qmark style used with executemany():
lang_list = [
    ("Fortran", 1957),
    ("Python", 1991),
    ("Go", 2009),
]
cur.executemany("INSERT INTO LANG VALUES (?, ?)", lang_list)

# And this is the named style:
cur.execute("SELECT * FROM LANG WHERE FIRST_APPEARED=:year", {"year": 1972})
rows = cur.fetchall()
print(rows)

df = pd.read_sql("SELECT * FROM LANG WHERE FIRST_APPEARED=1972", con)

con.close()

df


[('C', 1972)]


,NAME,FIRST_APPEARED
0,C,1972


## APScheduler 모듈
- Advaned Python Scheduler의 약자
- 파이썬 코드를 주기적으로 실행
- 작업 기간, 일시 정지, 다시 시작, 삭제 등이 가능함
- MongoDB,

In [1]:
!pip install apscheduler

  Using cached tzlocal-2.1-py2.py3-none-any.whl (16 kB)


## 세 가지 수행 방식
- Interval 방식 : 일정 주기마다 코드 수행
- Date 방식 : 특정 날짜에 코드 수행
- Cron 방식 : Cron 표현식으로 코드 수행

## 스케줄러의 종류
- BlockingScheduler: 단일 작업 수행시 사용
- BackgroundScheduler : 다수 작업 수행시 사용

## 사용법
1. 스케줄러 객체 생성
2.
3.

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler

def job():
    print('Hello World!')
    
sched = BlockingScheduler()

sched.add_job(job, 'interval'
              ,start_date='2021-06-02 14:31:00'
              ,end_date='2021-06-02 14:32:00'
              ,seconds=3) # 매 3초 간격으로 실행

print('Start')
sched.start()
print('End')

Start
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!


## date 방식 사용법
- 특정 시각

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
import datetime

def job(*args):
    print(args)
    
sched = BlockingScheduler()

sched.add_job(job, 'date'
              , run_date = datetime.datetime(2021,6,2,14,39,0)
              , args=['abc'])

print('Start')
sched.start()
print('End')

Start
('abc',)


## Cron 사용법

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler

def job():
    print('test')
    
shced = BlockingScheduler()

shced.add_job(job,'cron',month = '6-8,11-12', day = 2
              , hour='14-15', minute=46, second=20)

print('Start')
shced.start()
print('End')

Start
test


## 네이버 뉴스 수집 및 저장 프로그램 스케줄링
1. 웹 스크래핑
2. DB 저장
3. 스케줄 등록

In [8]:
# BeautifulSoup 으로 뉴스 헤드라인 기사 제목과 링크를 수집해보세요
import requests
from bs4 import BeautifulSoup
import datetime

url = 'https://news.naver.com'

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'
}

res = requests.get(url, headers=headers)
print(res.status_code)

bs = BeautifulSoup(res.text, 'lxml')

ul = bs.select_one('#today_main_news > div.hdline_news > ul')
lis = ul.select('li')
news_list = [] # 뉴스 기사를 담을 리스트
for li in lis:
    title = li.select_one('div.hdline_article_tit > a').get_text().strip()
    link = url + li.select_one('div.hdline_article_tit > a').get('href', '')
#     print(title, link)
    
    news_list.append((datetime.datetime.now(),title, link))
    
print(news_list)
    

200
[(datetime.datetime(2021, 6, 3, 14, 28, 49, 318012), '이용구 “1000만원 합의금, 폭행영상 삭제 대가 아냐”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=005&aid=0001444533'), (datetime.datetime(2021, 6, 3, 14, 28, 49, 318012), '文대통령 "절망스러웠을 피해자 생각하면 가슴 아프다"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=079&aid=0003511504'), (datetime.datetime(2021, 6, 3, 14, 28, 49, 319018), "6월 모평 국어 작년 수능과 난이도 비슷…'언어와매체' 더 어려워", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=277&aid=0004913993'), (datetime.datetime(2021, 6, 3, 14, 28, 49, 319018), "'동학·서학개미 생큐'…증권사 1분기 순익 3조 '사상 최대'", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=421&aid=0005391189'), (datetime.datetime(2021, 6, 3, 14, 28, 49, 319018), '대구시 화이자백신 사기당할 뻔? 정부 "정품 아니다. 국제수사 협력"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=469&aid=0000608522')]


## DB 저장

In [25]:
import sqlite3 as sq


con = sq.connect("naver_news.db")
cur = con.cursor()


# cur.execute("DROP TABLE NAVER_NEWS")

cur.execute("CREATE TABLE IF NOT EXISTS NAVER_NEWS (REG_DATE TEXT, TITLE TEXT, LINK TEXT)")

con.close()

In [5]:
cur.execute('SELECT * FROM NAVER_NEWS')

In [24]:
import sqlite3 as sq
import pandas as pd

con = sq.connect("naver_news.db")
cur = con.cursor()

cur.executemany('''
                INSERT INTO NAVER_NEWS(REG_DATE, TITLE, LINK) VALUES(?,?,?)
                '''
               , news_list
               )

con.commit()

df = pd.read_sql('SELECT*FROM NAVER_NEWS', con)
df = df.drop_duplicates(subset='TITLE',keep='last')

con.close()

df

,REG_DATE,TITLE,LINK
15,2021-06-03 14:28:49.318012,"이용구 “1000만원 합의금, 폭행영상 삭제 대가 아냐”",https://news.naver.com/main/read.nhn?mode=LSD&...
16,2021-06-03 14:28:49.318012,"文대통령 ""절망스러웠을 피해자 생각하면 가슴 아프다""",https://news.naver.com/main/read.nhn?mode=LSD&...
17,2021-06-03 14:28:49.319018,6월 모평 국어 작년 수능과 난이도 비슷…'언어와매체' 더 어려워,https://news.naver.com/main/read.nhn?mode=LSD&...
18,2021-06-03 14:28:49.319018,'동학·서학개미 생큐'…증권사 1분기 순익 3조 '사상 최대',https://news.naver.com/main/read.nhn?mode=LSD&...
19,2021-06-03 14:28:49.319018,"대구시 화이자백신 사기당할 뻔? 정부 ""정품 아니다. 국제수사 협력""",https://news.naver.com/main/read.nhn?mode=LSD&...


In [28]:
import requests
from bs4 import BeautifulSoup
import datetime
import sqlite3 as sq
import pandas as pd
import schedule
import time

def scrap():
    url = 'https://news.naver.com'

    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'
    }

    res = requests.get(url, headers=headers)
    print(res.status_code)

    bs = BeautifulSoup(res.text, 'lxml')

    ul = bs.select_one('#today_main_news > div.hdline_news > ul')
    lis = ul.select('li')
    news_list = [] # 뉴스 기사를 담을 리스트
    for li in lis:
        title = li.select_one('div.hdline_article_tit > a').get_text().strip()
        link = url + li.select_one('div.hdline_article_tit > a').get('href', '')
    #     print(title, link)

        news_list.append((datetime.datetime.now(),title, link))

    print(news_list)
    return news_list

def insert_db(news):

    con = sq.connect("naver_news.db")
    cur = con.cursor()

    cur.executemany('''
                    INSERT INTO NAVER_NEWS(REG_DATE, TITLE, LINK) VALUES(?,?,?)
                    '''
                   , news
                   )

    con.commit()
    con.close()
def create_table():
    con = sq.connect("naver_news.db")
    cur = con.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS NAVER_NEWS (REG_DATE TEXT, TITLE TEXT, LINK TEXT)")

con.close()
def job():
    news = scrap()
    insert_db(news)
    
create_table()

schedule.every(5).seconds.do(job)
schedule.every(10).seconds.do(scrap)
while True:
    schedule.run_pending()
    time.sleep(5)
    

200
[(datetime.datetime(2021, 6, 3, 15, 10, 4, 94575), '이용구 "합의금 1천만원, 블랙박스 영상삭제 대가 아냐"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=003&aid=0010531199'), (datetime.datetime(2021, 6, 3, 15, 10, 4, 94575), '전국에 비…밤에 대부분 그쳐요[오후날씨 꿀팁]', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=103&oid=056&aid=0011056586'), (datetime.datetime(2021, 6, 3, 15, 10, 4, 94575), '윤건영 “올림픽 보이콧 생각해야” VS 최문순 “日 전술에 말려드는 것”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=022&aid=0003587146'), (datetime.datetime(2021, 6, 3, 15, 10, 4, 95574), "국방부 '군검찰 수사심의위' 설치…부사관 사망 사건부터 적용", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=421&aid=0005391225'), (datetime.datetime(2021, 6, 3, 15, 10, 4, 95574), '문대통령 "진보, 단합할 때 지지 만들어져"…내부 결속 강조(종합)', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=421&aid=0005391618')]
200
[(datetime.datetime(2021, 6, 3, 15, 10, 4, 230203), '이용구 "합의금 1천만원, 블랙박스 영상삭제 대가 아냐"', 'https://news.nave

200
[(datetime.datetime(2021, 6, 3, 15, 10, 25, 912222), '"어젠 그냥 여름이던데 오늘은 시원하려나"[날씨톡톡]', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=103&oid=422&aid=0000488170'), (datetime.datetime(2021, 6, 3, 15, 10, 25, 912222), '이용구 "합의금 1천만원, 블랙박스 영상삭제 대가 아냐"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=003&aid=0010531199'), (datetime.datetime(2021, 6, 3, 15, 10, 25, 912222), '공정위, 삼성웰스토리 동의의결 기각', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=011&aid=0003918630'), (datetime.datetime(2021, 6, 3, 15, 10, 25, 912222), '文 "성추행 피해 부사관의 절망, 가슴 아파… 엄정 수사·조치하라"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=469&aid=0000608514'), (datetime.datetime(2021, 6, 3, 15, 10, 25, 912222), '[현장영상] 與 초선의원 68명, 대통령과 간담회...어떤 이야기 오갔나', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=052&aid=0001596403')]
200
[(datetime.datetime(2021, 6, 3, 15, 10, 26, 54395), '"어젠 그냥 여름이던데 오늘은 시원하려나"[날씨톡톡]', 'https://news.naver.

200
[(datetime.datetime(2021, 6, 3, 15, 10, 42, 598102), '‘네이버 통유리 사옥’ 태양반사광 주민피해 소송… 대법 “다시 재판해야”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=366&aid=0000727366'), (datetime.datetime(2021, 6, 3, 15, 10, 42, 598102), '정부 “대구시 도입하려는 화이자 백신 진위 의심… 추진 않기로”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=022&aid=0003587149'), (datetime.datetime(2021, 6, 3, 15, 10, 42, 598102), '국회의원 132명, 일본 올림픽 페이지 독도 표기 규탄 성명', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=025&aid=0003106781'), (datetime.datetime(2021, 6, 3, 15, 10, 42, 598102), '文대통령 "백신 접종 속도 내고 있어, 집단면역시기 당겨질 것"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=016&aid=0001843873'), (datetime.datetime(2021, 6, 3, 15, 10, 42, 599100), '6000억 넘는 M&A, 공정위 심사 받는다…소규모 비상장사는 공시의무 면제', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=366&aid=0000727404')]
200
[(datetime.datetime(2021, 6, 3, 15, 10, 42, 752116), '‘네이버 통유리 사옥’ 태양반사광 주민피

[(datetime.datetime(2021, 6, 3, 15, 10, 59, 509800), '정부 "대구시 주선 화이자 백신 진위 의심…구매 안한다"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=001&aid=0012435457'), (datetime.datetime(2021, 6, 3, 15, 10, 59, 509800), '‘네이버 통유리 사옥’ 태양반사광 주민피해 소송… 대법 “다시 재판해야”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=366&aid=0000727366'), (datetime.datetime(2021, 6, 3, 15, 10, 59, 509800), '문 대통령, ‘공군 부사관 성추행’ 사건에 엄정 수사 지시…“가슴 아파”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=025&aid=0003106795'), (datetime.datetime(2021, 6, 3, 15, 10, 59, 510800), '이용구 법무차관 “택시기사 준 1000만원, 영상 삭제 대가 아닌 합의금”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=015&aid=0004556629'), (datetime.datetime(2021, 6, 3, 15, 10, 59, 510800), '군검찰 `부사관 성추행` 원점서 수사…“회유·은폐 가담자 모두 소환”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=018&aid=0004946285')]
200
[(datetime.datetime(2021, 6, 3, 15, 10, 59, 656790), '[속보]文대통령 "성추행 피해 부사관,

200
[(datetime.datetime(2021, 6, 3, 15, 11, 21, 401832), "'반사광 최대 2.9만배' 네이버 유리사옥 법정다툼, 주민들 이겼다", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=008&aid=0004596631'), (datetime.datetime(2021, 6, 3, 15, 11, 21, 401832), '공군 부사관 유족측, 3명 추가 고소…“성추행 피해 더있다”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=032&aid=0003077969'), (datetime.datetime(2021, 6, 3, 15, 11, 21, 401832), '이용구 "택시기사, 억울하게 증거인멸죄로 입건돼 죄송"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=055&aid=0000898333'), (datetime.datetime(2021, 6, 3, 15, 11, 21, 401832), '문 대통령 “경제 성과에도 국민 고통에 마음 아파”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=015&aid=0004556729'), (datetime.datetime(2021, 6, 3, 15, 11, 21, 401832), '화이자 백신 접종하는 어르신들', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=421&aid=0005391556')]
200
[(datetime.datetime(2021, 6, 3, 15, 11, 21, 614570), '文대통령, 공군 성폭력피해사건 “최고 상급자 등 지휘라인 문제 살펴야”', 'https://news.naver.c

...

200
[(datetime.datetime(2021, 6, 3, 15, 26, 41, 493740), '문대통령 "진보, 단합할 때 지지 만들어져"…내부 결속 강조(종합)', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=421&aid=0005391618'), (datetime.datetime(2021, 6, 3, 15, 26, 41, 493740), '\'폭행 혐의\' 이용구 "1000만원은 합의금, 영상삭제 대가 아냐"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=015&aid=0004556555'), (datetime.datetime(2021, 6, 3, 15, 26, 41, 493740), '제주도, 그린수소로 수소전기차 보급도 촉진[이슈분석]', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=103&oid=030&aid=0002948630'), (datetime.datetime(2021, 6, 3, 15, 26, 41, 493740), '文대통령, 與 초선들과 간담회 “국민 고통 마음 아파…혁신과 역동성 발휘해야”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=277&aid=0004914066'), (datetime.datetime(2021, 6, 3, 15, 26, 41, 493740), "[속보] 화이자 접종 완료 후 확진→ '돌파감염' 5명 추가… 누적 9명", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=082&aid=0001095866')]
200
[(datetime.datetime(2021, 6, 3, 15, 26, 41, 631945), '靑 “문 대통령, 女부사관 사건에 

200
[(datetime.datetime(2021, 6, 3, 15, 27, 3, 169833), '"동학개미가 먹여 살렸다"…증권사, 증시 호황에 순익 \'사상 최대\'', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=417&aid=0000700575'), (datetime.datetime(2021, 6, 3, 15, 27, 3, 169833), '문대통령 "경제 성과 있으나 국민 고통…마음 아프다"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=001&aid=0012435548'), (datetime.datetime(2021, 6, 3, 15, 27, 3, 169833), "공정위, 삼성웰스토리 부당지원 '자진시정' 신청 기각", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=001&aid=0012435682'), (datetime.datetime(2021, 6, 3, 15, 27, 3, 169833), '화이자 "대구시 제안받은 백신 정품 아닌 듯"…정부 "도입 협의 안 해"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=003&aid=0010531759'), (datetime.datetime(2021, 6, 3, 15, 27, 3, 170839), '“국어 선택과목, 언어와 매체가 더 어려웠다”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=018&aid=0004946417')]
200
[(datetime.datetime(2021, 6, 3, 15, 27, 3, 312908), '"동학개미가 먹여 살렸다"…증권사, 증시 호황에 순익 \'사상 최대\'', 'https://

200
[(datetime.datetime(2021, 6, 3, 15, 27, 20, 57112), "文대통령, '공군 여부사관 성폭력 사망사건' 엄정수사 강력 지시", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=123&aid=0002247485'), (datetime.datetime(2021, 6, 3, 15, 27, 20, 57112), '이용구 "택시기사에 거듭 사과...영상 유포 우려해 삭제 요청"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=052&aid=0001596289'), (datetime.datetime(2021, 6, 3, 15, 27, 20, 57112), "SK 이번엔 '청록수소' 투자, 韓 수소 선택지 늘어난다", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=008&aid=0004596584'), (datetime.datetime(2021, 6, 3, 15, 27, 20, 57112), "국방부, 여성 부사관 성추행·사망 사건에 '군검찰 수사심의위' 설치", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=047&aid=0002315027'), (datetime.datetime(2021, 6, 3, 15, 27, 20, 57112), '1분기 증권사 순이익, 분기 사상 최대…3조 육박', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=003&aid=0010531767')]
200
[(datetime.datetime(2021, 6, 3, 15, 27, 20, 245228), "文대통령, '공군 여부사관 성폭력 사망사건' 엄정수사 강력 지시", 'https://

200
[(datetime.datetime(2021, 6, 3, 15, 27, 36, 986357), '대법 "네이버 통유리 사옥, 태양반사광 생활 방해 인정돼"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=011&aid=0003918547'), (datetime.datetime(2021, 6, 3, 15, 27, 36, 986357), '尹측 “무리한 가족 수사 1년 3개월 째, 정치인들 언행 도 넘어”', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=023&aid=0003617946'), (datetime.datetime(2021, 6, 3, 15, 27, 36, 986357), '문대통령 "국민 고통 마음아파"…與초선 "곳간 더 열어야"', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=422&aid=0000488200'), (datetime.datetime(2021, 6, 3, 15, 27, 36, 986357), "'보복운전' 구본성 아워홈 부회장 1심서 집행유예 2년", 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=009&aid=0004803679'), (datetime.datetime(2021, 6, 3, 15, 27, 36, 987356), '금융위, "공매도 원활히 안착"…무차입 의심거래 720건 감리(종합)', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=277&aid=0004914092')]


KeyboardInterrupt: 